In [19]:
import numpy as np
import time
import cosmocnc as cnc
import os

# Set up configuration

In [20]:
# --------------------- Config ---------------------
# 10 deg x 10 deg patch: (10/180*pi) x (10/180*pi) steradians
# Full sky = 4*pi steradians
sky_area_deg2 = 10 * 10
full_sky_deg2 = 4*np.pi*(180/np.pi)**2
sky_frac = sky_area_deg2 / full_sky_deg2
sky_frac = 1.0
get_sky_coords = False

# Two parameter files: catalogue_sbi_0.csv and catalogue_sbi_1.csv
indices = [0, 1]
param_dir = "../catalogue_demo"
param_tpl = "params_sbi_{idx}.csv"

# One catalogue per run of catalogue_generator
n_catalogues = 1
# Where to save generated catalogues
save_tpl = "../catalogue_generated/catalogue_test_{idx}.npy"


# Initialization (note the precision parameters, can adjust to low precision parameters for testing)

In [21]:
cnc_params = {
    "number_cores_hmf": 1,
    # "n_points": 1024*64,
    # "n_z": 10000,
    "n_points": 1024*16,
    "n_z": 1000,
    "z_min": 0.005,
    "z_max": 3.0,
    "M_min": 1e14,
    "M_max": 1e16,
    "cosmo_model": "lcdm",
    "hmf_type": "Tinker08",
    "mass_definition": "500c",
    "cosmology_tool": "classy_sz",
    "hmf_calc": "cnc",
    "hmf_type_deriv": "numerical",
    "power_spectrum_type": "cosmopower",
    "cosmo_amplitude_parameter": "logA",
    "Hubble_parameter": "h",
    "cosmo_param_density": "physical",
    "interp_tinker": "log",
    "class_sz_cosmo_model": "lcdm",
    "cosmocnc_verbose": "none",
    "load_catalogue": False,
    "survey_sr": "/Users/licongxu/csd3/cosmocnc/surveys/survey_sr_so_sim.py",
    "survey_cat": "/Users/licongxu/csd3/cosmocnc/surveys/survey_cat_so_sim.py",
    "class_sz_ndim_masses": 100,
    "class_sz_ndim_redshifts": 500,
    "class_sz_concentration_parameter": "B13",
    "class_sz_output": "mPk,m500c_to_m200c,m200c_to_m500c",
    "class_sz_hmf": "T08M500c",
    "class_sz_use_m500c_in_ym_relation": 1,
    "class_sz_use_m200c_in_ym_relation": 0,
    "observables": [["q_so_sim"]],
    "obs_select": "q_so_sim",
    "stacked_likelihood": False,
    "obs_select_min": 0.0,
    "obs_select_max": 0.0,
}

# Baseline cosmology; will be overwritten per-catalogue from CSV
cosmo_params = {
    "h": 0.6766,
    "Ob0h2": 0.02242,
    "Oc0h2": 0.1193,
    "logA": 3.047,
    "n_s": 0.9665,
    "m_nu": 0.06,
    "tau_reio": 0.0544,
}

scal_rel_params = cnc.scaling_relation_params_default
scal_rel_params["bias_sz"] = 0.8
scal_rel_params["dof"] = 0.0
# --------------------------------------------------

In [22]:
# Initialise number counts once
number_counts = cnc.cluster_number_counts(cnc_params=cnc_params)
number_counts.cosmo_params = cosmo_params
number_counts.scal_rel_params = scal_rel_params
number_counts.initialise()
print("Initialised number_counts.")

Initialised number_counts.


In [23]:
def load_params_from_csv(path):
    """
    Load one row with header:
    h,logA,n_s,Ob0h2,Oc0h2,B,A_cib,A_ir,A_rs
    Returns a dict with only the first five relevant fields.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Parameter file not found: {path}")

    data = np.genfromtxt(path, delimiter=",", names=True)
    # Ensure we can index like an array even if there's only one row
    if data.shape == ():  # single row -> np.void
        row = data
    else:
        if data.shape[0] < 1:
            raise ValueError(f"No rows in parameter file: {path}")
        row = data[0]

    return {
        "h": float(row["h"]),
        "logA": float(row["logA"]),
        "n_s": float(row["n_s"]),
        "Ob0h2": float(row["Ob0h2"]),
        "Oc0h2": float(row["Oc0h2"]),
    }

# Generate cluster catalogues from parameters
### Note: use classy_sz post23 here

In [24]:
t_global_start = time.time()

for idx in indices:
    print(f"\nCatalogue index: {idx}")
    param_path = os.path.join(param_dir, param_tpl.format(idx=idx))
    params = load_params_from_csv(param_path)
    print("Parameters from CSV:", params)

    # Update cosmology from the CSV (ignore the last four irrelevant params)
    cosmo_params["h"] = params["h"]
    cosmo_params["logA"] = params["logA"]
    cosmo_params["n_s"] = params["n_s"]
    cosmo_params["Ob0h2"] = params["Ob0h2"]
    cosmo_params["Oc0h2"] = params["Oc0h2"]

    number_counts.update_params(cosmo_params, scal_rel_params)

    # Reproducible seed per catalogue
    unique_seed = 20000401 + int(idx)
    np.random.seed(unique_seed)

    t0 = time.time()
    print("Generating Catalogue", idx)
    cat_gen = cnc.catalogue_generator(
        number_counts=number_counts,
        n_catalogues=n_catalogues,
        seed=unique_seed,
        get_sky_coords=get_sky_coords,
        sky_frac=sky_frac,
    )
    cat_gen.generate_catalogues_hmf()
    catalogue_list = cat_gen.catalogue_list
    catalogue = catalogue_list[0]

    M = catalogue["M"]
    z = catalogue["z"]
    print("N clusters:", len(M))
    print("Time generation (s):", time.time() - t0)

    np.save(save_tpl.format(idx=idx), catalogue_list)

t_total = time.time() - t_global_start
print(f"\n=== Finished {len(indices)} catalogues in {t_total:.2f} s ({t_total/60:.1f} min) ===")


Catalogue index: 0
Parameters from CSV: {'h': 0.65003166, 'logA': 2.9999497, 'n_s': 0.94439447, 'Ob0h2': 0.0218048, 'Oc0h2': 0.11680297}
Generating Catalogue 0
Sky frac 1.0
N clusters: 331999
Time generation (s): 7.160011053085327

Catalogue index: 1
Parameters from CSV: {'h': 0.89233252, 'logA': 2.87202526, 'n_s': 0.94435259, 'Ob0h2': 0.02449334, 'Oc0h2': 0.12588283}
Generating Catalogue 1
Sky frac 1.0
N clusters: 63927
Time generation (s): 7.164183139801025

=== Finished 2 catalogues in 24.85 s (0.4 min) ===
